# Cut-in detection

In this notebook, the cut ins are detected. Furthermore, the detected cut ins are written to a database using the domain model.

Start the emulator.

    netsh http add sslcert ipport=0.0.0.0:8081 appid={b4fb8546-7c4c-44bb-ac5b-9134d4f81626} certhash=<THUMBPRINT>

Where `<THUMBPRINT>` can be found when going to the certificates (Manage computer certificates).
    
    netsh http add sslcert ipport=0.0.0.0:8081 appid={b4fb8546-7c4c-44bb-ac5b-9134d4f81626} certhash=95da5ca7196af4932259afba1f22507e1ddab936



In [ ]:
from domain_model.document_management import DocumentManagement as dm
import pydocumentdb
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
URI = "https://localhost:8081"
PRIMARY_KEY = \
    "C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=="

In [ ]:
DM = dm(URI, PRIMARY_KEY)
client = DM.client

In [ ]:
DM.ids["database_id"] = "test"
DM.ids["collection_ids"] = "listofid"
DM.ids["ActorCategory"] = "actorcat"
DM.ids["ActivityCategory"] = "activitycat1"
DM.ids["StaticEnvironmentCategory"] = "staticenvcat"
DM.ids["ScenarioCategory"] = "scencat1"
DM.ids["Actor"] = "actor123"
DM.ids["Activity"] = "activity"
DM.ids["StaticEnvironment"] = "staticenv123"
DM.ids["Scenario"] = "scenario"

In [ ]:
try:
    database = client.CreateDatabase(dict(id=DM.ids["database_id"]))
except pydocumentdb.errors.HTTPFailure as e:
    if e.status_code == 409:
        print("Database already exist")
    else:
        raise e

In [ ]:
collections = [DM.ids["collection_ids"], 
               DM.ids["ActorCategory"], DM.ids["ActivityCategory"],
               DM.ids["StaticEnvironmentCategory"], DM.ids["ScenarioCategory"],
               DM.ids["Actor"], DM.ids["Activity"], DM.ids["StaticEnvironment"], 
               DM.ids["Scenario"]]
for collection in collections:
    try:
        client.CreateCollection("dbs/test", dict(id=collection))
    except pydocumentdb.errors.HTTPFailure as e:
        if e.status_code == 409:
            print("Collection '{:s}' already exist".format(collection))
        else:
            raise e

In [ ]:
for collection in collections:
    try:
        last_id = DM.get_last_id(collection)
        print("Last id for '{:s}': {:d}".format(collection, last_id))
    except pydocumentdb.errors.HTTPFailure as e:
        if e.status_code != 404:
            raise e

        client.CreateDocument("dbs/" + DM.ids["database_id"] + "/colls/" + DM.ids["collection_ids"],
                              dict(id=collection, value=0))

In [ ]:
x = client.GetDatabaseAccount(url_connection=URI)

In [ ]:
import binascii

In [ ]:
binascii.a2b_base64("listofid")